In [14]:
import pandas as pd
import numpy as np
# # set up dictionaries of parameter options for each learner
param_opts = {
    'DecisionTreeClassifier': {},
    'GradientBoostingClassifier': {},
    'KNeighborsClassifier': {},
    'LogisticRegression': {},
    'RandomForestClassifier': {},
    'SVC': {}
}
ml_p = pd.read_csv('ml_p_options.csv')
ml_p.rename(columns={'alg_name':'classifier'},inplace=True)
for ml, df_ml in ml_p.groupby('classifier'):
    for p, df_ml_p in df_ml.groupby('parameters'):
        d = eval(p)
        for keys,v in d.items():
            if keys not in param_opts[ml].keys():
                param_opts[ml][keys] = [v]
            elif v not in param_opts[ml][keys]:
                param_opts[ml][keys].append(v)
print(param_opts)

{'DecisionTreeClassifier': {'criterion': ['entropy', 'gini'], 'max_depth': [2, 4, 6, 8], 'min_samples_leaf': [10, 20, 5], 'min_samples_split': [10, 20, 5]}, 'GradientBoostingClassifier': {'learning_rate': [0.01, 0.1, 1], 'max_depth': [1, 10, 5], 'max_features': ['log2', 'sqrt'], 'min_samples_leaf': [10, 20, 5], 'min_samples_split': [10, 20, 5], 'n_estimators': [100, 250], 'subsample': [0.5, 1]}, 'KNeighborsClassifier': {'n_neighbors': [1, 11, 3, 5, 7, 9], 'p': [1, 2], 'weights': ['distance', 'uniform']}, 'LogisticRegression': {'C': [0.0001, 0.001, 0.01, 0.1, 0.5, 1, 10, 25], 'dual': ['false', 'true'], 'penalty': ['l1', 'l2']}, 'RandomForestClassifier': {'bootstrap': ['false', 'true'], 'criterion': ['entropy', 'gini'], 'max_features': ['log2', 'sqrt'], 'min_samples_leaf': [10, 20, 5], 'min_samples_split': [10, 20, 5], 'n_estimators': [100, 250]}, 'SVC': {'C': [0.0001, 0.001, 0.01, 0.1, 0.5, 1, 10, 25], 'kernel': ['poly', 'rbf'], 'tol': [0.0001, 0.001, 0.01, 0.1, 1e-05]}}


In [15]:
import pandas as pd
import numpy as np

data = pd.read_csv('hibachi_combined_results_correct_datasets.csv.gz') 
print(data.head())
data['accuracy'] = data['accuracy'].apply(lambda x: round(x, 3))
print('loaded ',data['dataset'].unique().shape[0],'datasets and ', data['classifier'].unique().shape[0],'classifiers')
# subset data to classifiers used in PennAI
# pennai_classifiers = ['LogisticRegression', 'RandomForestClassifier', 'SVC', 
#                       'KNeighborsClassifier', 'DecisionTreeClassifier', 'GradientBoostingClassifier']
# mask = np.array([c in pennai_classifiers for c in data['classifier'].values])
# data = data.loc[mask,:]
print('datasets (',len(data['dataset'].unique()),')')
print('classifiers (',len(data['classifier'].unique()),'):',data['classifier'].unique())
for ml, df_g in data.groupby('classifier'):
    print('parameters for ',ml,'(',len(df_g['parameters'].unique()),'):',df_g['parameters'].unique()[:5])


                                       dataset              classifier  \
0  AAA_bench_mod_4_up_down_dt_gb_score_0-36112  DecisionTreeClassifier   
1  AAA_bench_mod_4_up_down_dt_gb_score_0-36112  DecisionTreeClassifier   
2  AAA_bench_mod_4_up_down_dt_gb_score_0-36112  DecisionTreeClassifier   
3  AAA_bench_mod_4_up_down_dt_gb_score_0-36112  DecisionTreeClassifier   
4  AAA_bench_mod_4_up_down_dt_gb_score_0-36112  DecisionTreeClassifier   

                                          parameters  accuracy   macrof1  \
0  min_impurity_decrease=0.0,max_features=0.1,cri...     0.549  0.548899   
1  min_impurity_decrease=0.0,max_features=0.1,cri...     0.554  0.554000   
2  min_impurity_decrease=0.0,max_features=0.25,cr...     0.567  0.566989   
3  min_impurity_decrease=0.0,max_features=0.25,cr...     0.556  0.555956   
4  min_impurity_decrease=0.0,max_features=0.5,cri...     0.751  0.750868   

   bal_accuracy  
0         0.549  
1         0.554  
2         0.567  
3         0.556  
4       

In [16]:
from collections import OrderedDict

def convert_params(params):
    """convert from sklearn-style parameter formatting to dictionary, PennAI style."""
    pdict = {}
    for kv in params.split(','):
        if len(kv)==0: continue
        if 'random_state' in kv: continue
#         print(kv.split('='))
        pdict[str(kv.split('=')[0])] = kv.split('=')[1]
    for k,v in pdict.items():
        try:
            pdict[k] = int(v)
        except ValueError:
            try:     
                pdict[k] = float(v)
            except ValueError:
                pass

    return OrderedDict(sorted(pdict.items()))


data['parameters'] = data['parameters'].apply(lambda x: convert_params(x))

In [11]:
for ml, df_g in data.groupby('classifier'):
    print('example parameters for ',ml,'(',len(df_g['parameters'].apply(str).unique()),'):',
           df_g['parameters'].apply(str).unique()[0])

example parameters for  DecisionTreeClassifier ( 280 ): OrderedDict([('criterion', 'gini'), ('max_features', 0.1), ('min_impurity_decrease', 0.0)])
example parameters for  GradientBoostingClassifier ( 450 ): OrderedDict([('learning_rate', 0.01), ('loss', 'deviance'), ('max_features', 'sqrt'), ('min_impurity_decrease', 0.0), ('n_estimators', 10)])
example parameters for  LogisticRegression ( 240 ): OrderedDict([('C', 0.5), ('dual', 'False'), ('fit_intercept', 'True'), ('penalty', 'l1')])
example parameters for  RandomForestClassifier ( 90 ): OrderedDict([('criterion', 'gini'), ('max_features', 'sqrt'), ('min_impurity_decrease', 0.0), ('n_estimators', 10)])
example parameters for  SVC ( 1188 ): OrderedDict([('C', 0.01), ('coef0', 0.0), ('degree', 2), ('gamma', 0.01), ('kernel', 'poly')])


In [ ]:
def valid_param_combo(ml,params):
#     print('ml',ml,type(ml).__name__)
#     print('params',params,type(params).__name__)

    for k,v in params.items():
        if k in param_opts[ml].keys():
            if param_opts[ml][k] is int:
                try:
                    if int(v) in param_opts[ml][k]:
                        return True
                except: 
                    return False
            elif param_opts[ml][k] is float:
                try:
                    if float(v) in param_opts[ml][k]:
                        return True
                except: 
                    return False
            elif v not in param_opts[ml][k]:
#                 if ml == 'LogisticRegression' and k=='C':
#                     print('eliminating',params,'for',ml,'with',k,':',v)
                    return False
    return True

mask = [valid_param_combo(row['classifier'],row['parameters']) for _, row in data.iterrows()]
mask = mask | data.classifier=='LogisticRegression'
data_filtered = data.loc[mask]

# data_filtered = data.loc[lambda x: valid_param_combo(i['classifier'],i['parameters']) for i in x]


eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l1')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'True'), ('penalty', 'l1')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'True'), ('fit_intercept', 'False'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'True'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'True'), ('fit_intercept', 'True'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 10.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l1')]) for LogisticRegression with C : 10.5
eliminating OrderedDict

eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l1')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'True'), ('penalty', 'l1')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'True'), ('fit_intercept', 'False'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'True'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'True'), ('fit_intercept', 'True'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 10.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l1')]) for LogisticRegression with C : 10.5
eliminating OrderedDict

eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l1')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'True'), ('penalty', 'l1')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'True'), ('fit_intercept', 'False'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'True'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'True'), ('fit_intercept', 'True'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 10.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l1')]) for LogisticRegression with C : 10.5
eliminating OrderedDict

eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l1')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'True'), ('penalty', 'l1')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'True'), ('fit_intercept', 'False'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'True'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'True'), ('fit_intercept', 'True'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 10.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l1')]) for LogisticRegression with C : 10.5
eliminating OrderedDict

eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l1')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'True'), ('penalty', 'l1')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'True'), ('fit_intercept', 'False'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'True'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'True'), ('fit_intercept', 'True'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 10.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l1')]) for LogisticRegression with C : 10.5
eliminating OrderedDict

...

eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l1')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'True'), ('penalty', 'l1')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'True'), ('fit_intercept', 'False'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'True'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'True'), ('fit_intercept', 'True'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 10.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l1')]) for LogisticRegression with C : 10.5
eliminating OrderedDict

eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l1')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'True'), ('penalty', 'l1')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'True'), ('fit_intercept', 'False'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'True'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'True'), ('fit_intercept', 'True'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 10.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l1')]) for LogisticRegression with C : 10.5
eliminating OrderedDict

eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l1')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'True'), ('penalty', 'l1')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'True'), ('fit_intercept', 'False'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'True'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'True'), ('fit_intercept', 'True'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 10.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l1')]) for LogisticRegression with C : 10.5
eliminating OrderedDict

eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l1')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'True'), ('penalty', 'l1')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'True'), ('fit_intercept', 'False'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'True'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'True'), ('fit_intercept', 'True'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 10.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l1')]) for LogisticRegression with C : 10.5
eliminating OrderedDict

eliminating OrderedDict([('C', 9.5), ('dual', 'True'), ('fit_intercept', 'False'), ('penalty', 'l2')]) for LogisticRegression with C : 9.5
eliminating OrderedDict([('C', 9.5), ('dual', 'False'), ('fit_intercept', 'True'), ('penalty', 'l2')]) for LogisticRegression with C : 9.5
eliminating OrderedDict([('C', 9.5), ('dual', 'True'), ('fit_intercept', 'True'), ('penalty', 'l2')]) for LogisticRegression with C : 9.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l1')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'True'), ('penalty', 'l1')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([('C', 1.5), ('dual', 'True'), ('fit_intercept', 'False'), ('penalty', 'l2')]) for LogisticRegression with C : 1.5
eliminating OrderedDict([(

In [15]:
for ml, df_g in data_filtered.groupby('classifier'):
    print('example parameters for ',ml,'(',len(df_g['parameters'].apply(str).unique()),'):',
           df_g['parameters'].apply(str).unique()[0])

example parameters for  DecisionTreeClassifier ( 154 ): OrderedDict([('criterion', 'entropy'), ('max_features', 0.1), ('min_weight_fraction_leaf', 0.0)])
example parameters for  GradientBoostingClassifier ( 36 ): OrderedDict([('learning_rate', 0.01), ('loss', 'deviance'), ('max_depth', 1), ('max_features', 'log2'), ('n_estimators', 100)])
example parameters for  KNeighborsClassifier ( 12 ): OrderedDict([('n_neighbors', 1), ('weights', 'distance')])
example parameters for  RandomForestClassifier ( 44 ): OrderedDict([('criterion', 'entropy'), ('max_features', 'log2'), ('min_weight_fraction_leaf', 0.0), ('n_estimators', 100)])
example parameters for  SVC ( 600 ): OrderedDict([('C', 0.01), ('coef0', 0.0), ('degree', 2), ('gamma', 0.01), ('kernel', 'poly')])


In [8]:
param_opts['LogisticRegression']

{'C': [0.0001, 0.001, 0.01, 0.1, 0.5, 1, 10, 25],
 'dual': ['false', 'true'],
 'penalty': ['l1', 'l2']}

In [20]:
print(data[data['classifier']=='KNeighborsClassifier']['parameters'].apply(str).unique())

["{'criterion': 'gini', 'learning_rate': 50, 'kernel': 'sigmoid', 'weights': 'uniform', 'penalty': 'l2', 'loss': 'deviance', 'gamma': 50.0, 'degree': 2, 'min_weight_fraction_leaf': 0, 'coef0': 50.0, 'dual': 'True', 'fit_intercept': 'True', 'max_depth': 'None', 'max_features': 'sqrt', 'C': 0.01, 'n_estimators': 500, 'n_neighbors': 9}"]


In [44]:
data_filtered.head()

,dataset,classifier,parameters,accuracy,macrof1,bal_accuracy
175,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'criterion': 'entropy', 'max_features': 0.1, ...",0.507,0.507488,0.5075
176,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'criterion': 'gini', 'max_features': 0.1, 'mi...",0.476,0.476040,0.47625
177,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'criterion': 'entropy', 'max_features': 0.25,...",0.507,0.506832,0.506875
178,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'criterion': 'gini', 'max_features': 0.25, 'm...",0.490,0.489993,0.49
179,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'criterion': 'entropy', 'max_features': 0.5, ...",0.512,0.512497,0.5125


In [65]:
print('data_filtered shape: ', data_filtered.shape)
print('datasets (',len(data_filtered['dataset'].unique()),')')
print('classifiers (',len(data_filtered['classifier'].unique()),'):',data_filtered['classifier'].unique())
for ml, df_g in data_filtered.groupby('classifier'):
    print('parameters for ',ml,'(',len(df_g['parameters'].apply(str).unique()),'):',
           df_g['parameters'].apply(str).unique()[:5])

data_filtered shape:  (113740, 6)
datasets ( 165 )
classifiers ( 5 ): ['DecisionTreeClassifier' 'GradientBoostingClassifier'
 'KNeighborsClassifier' 'RandomForestClassifier' 'SVC']
parameters for  DecisionTreeClassifier ( 154 ): ["OrderedDict([('criterion', 'entropy'), ('max_features', 0.1), ('min_weight_fraction_leaf', 0.0)])"
 "OrderedDict([('criterion', 'gini'), ('max_features', 0.1), ('min_weight_fraction_leaf', 0.0)])"
 "OrderedDict([('criterion', 'entropy'), ('max_features', 0.25), ('min_weight_fraction_leaf', 0.0)])"
 "OrderedDict([('criterion', 'gini'), ('max_features', 0.25), ('min_weight_fraction_leaf', 0.0)])"
 "OrderedDict([('criterion', 'entropy'), ('max_features', 0.5), ('min_weight_fraction_leaf', 0.0)])"]
parameters for  GradientBoostingClassifier ( 36 ): ["OrderedDict([('learning_rate', 0.01), ('loss', 'deviance'), ('max_depth', 1), ('max_features', 'log2'), ('n_estimators', 100)])"
 "OrderedDict([('learning_rate', 0.01), ('loss', 'deviance'), ('max_depth', 1), ('max_f

In [18]:
clean_data = data
clean_data.loc[:, 'algorithm-parameters'] = (                                             
                                       clean_data['classifier'].values + '|' +                     
                                       clean_data['parameters'].apply(str).values) 
all_datasets = np.unique(clean_data['dataset'].values)
algp_toremove = []
for algp, group in clean_data.groupby('algorithm-parameters'):
    if (group['dataset'].count()<len(all_datasets)):
        #print(algp, '\n\t is missing results for', [d for d in all_datasets if d not in np.unique(group['dataset'])])
        #print('removing',algp)
        algp_toremove.append(algp)
        
mask = np.array([ap not in algp_toremove for ap in clean_data['algorithm-parameters'].values])
print('removing',np.sum(~mask),'algorithm-parameter combinations')
clean_data = clean_data.loc[mask,:]
print('new size:',len(clean_data))
clean_data.drop('algorithm-parameters',axis=1,inplace=True)

clean_data.groupby('classifier').count()

removing 22589 algorithm-parameter combinations
new size: 157462


/home/bill/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,dataset,parameters,accuracy,macrof1,bal_accuracy
classifier,,,,,
DecisionTreeClassifier,36680,36680,36680,36680,36680
GradientBoostingClassifier,58950,58950,58950,58950,58950
LogisticRegression,31440,31440,31440,31440,31440
RandomForestClassifier,11790,11790,11790,11790,11790
SVC,18602,18602,18602,18602,18602


In [19]:
for ml, df_g in data.groupby('classifier'):
    print('example parameters for ',ml,'(',len(df_g['parameters'].apply(str).unique()),'):',
           df_g['parameters'].apply(str).unique()[0])

NameError: name 'data_filtered' is not defined

# write modified data to file

In [20]:
print(clean_data.columns)
clean_data.to_csv('hibachi_combined_results_correct_datasets_mock_experiment.tsv.gz',compression='gzip',
            index=False,
            sep='\t',
            header=['dataset',
                     'algorithm',
                     'parameters',
                     'accuracy', 
                     'macrof1',
                     'bal_accuracy'])

Index(['dataset', 'classifier', 'parameters', 'accuracy', 'macrof1',
       'bal_accuracy'],
      dtype='object')
